# Word Generation In $G=\mathbb{Z}_2*\mathbb{Z}$

### Write $G=\langle x|x^2=1\rangle*\langle y \rangle$, with generating set $S=\{x,y,y^{-1}\}$.

Our interest is to generate all the **excursions** on the Cayley Graph of $G$ wrt $S$. These are words that end up back at one (Eg. $xx$ or $yy^{-1}$ or $yxxy^{-1}$). To avoid redundancy, we can generate **simple excursions**, those that don't go back to 1 "too early".

Let $Z_{g,X}$ be the set of words ($\in S*$) that evaluates to $g\in G$ and contains no proper prefix in $X\subseteq S$.
Let $Z_{g,X}^{(n)}$ be such words of length $n$.

Our Goal is to find $Z_{1,\emptyset}^{(n)}$ for some small $n$. Contenations of such words of varied $n$ gives another such word. To avoid redundency, We shall compute $Z_{1,\{1\}}^{(n)}$, and notice that 

$$Z_{1,\emptyset}^{(n)}=\left(\cup_{\lambda}[\times_{i}Z_{1,\{1\}}^{(\lambda_i)}]\right)\cup Z_{1,\{1\}}^{(n)}$$

where $\lambda=(\lambda_i)$ ranges over the $2^{n-1}-1$ nontrivial compostions of $n$. Furthermore, the $2^{n-1}$ sets involved in the union expression are pairwise disjoint.
